# 1.14b2: Black Hole Gradient Analysis

**Goal:** Why does the black hole move? Check if untrained tokens receive identical gradients.

## Hypothesis

The black hole (49 untrained tokens) moves because:

1. All tokens start at the same point (supermassive BH)
2. Forward pass: `logit[i] = h • W[i]` → identical logits for identical W[i]
3. Backward pass: gradient depends on logit
4. **Identical logits → identical gradients**
5. Identical gradients → tokens move together
6. Still coincident → cycle repeats

## Test

For untrained tokens at each timestep:
- Are the logits identical?
- Are the gradients identical?
- Does gradient equality → position equality?

## Expected Result

✓ Untrained tokens have identical (or near-identical) logits
✓ Untrained tokens have identical (or near-identical) gradients
✓ This explains why the black hole moves as a rigid body

## Parameters

In [1]:
# Training data to analyze
TRAINING_DATA_PATH = "../tensors/Lil_Gatsby/1.12b_training_data_sigma0e+00.safetensors"

# Corpus path (to identify untrained tokens)
CORPUS_PATH = "../data/gatsby_clean.txt"

# Analysis
STEPS_TO_CHECK = [0, 1, 2, 10, 100, 1000, 10000]  # Sample timesteps

## Imports

In [2]:
import torch
import numpy as np
from pathlib import Path
from safetensors.torch import load_file

print("✓ Imports complete")

✓ Imports complete


## Load Training Data

In [3]:
print(f"Loading training data: {TRAINING_DATA_PATH}\n")

data = load_file(TRAINING_DATA_PATH)
embeddings = data['embeddings'].float()  # (n_steps, vocab_size, hidden_dim)
gradients = data['grads'].float()        # (n_steps, vocab_size, hidden_dim)
logits = data['logits'].float()          # (n_steps, vocab_size)

n_steps, vocab_size, hidden_dim = embeddings.shape

print(f"✓ Loaded training data")
print(f"  Embeddings: {embeddings.shape}")
print(f"  Gradients: {gradients.shape}")
print(f"  Logits: {logits.shape}")

Loading training data: ../tensors/Lil_Gatsby/1.12b_training_data_sigma0e+00.safetensors

✓ Loaded training data
  Embeddings: torch.Size([10001, 128, 64])
  Gradients: torch.Size([10001, 128, 64])
  Logits: torch.Size([10001, 128])


## Identify Untrained Tokens

In [4]:
print(f"\nAnalyzing corpus: {CORPUS_PATH}\n")

with open(CORPUS_PATH, 'r', encoding='ascii') as f:
    corpus_text = f.read()

corpus_bytes = corpus_text.encode('ascii')
trained_tokens = sorted(set(corpus_bytes))
untrained_tokens = sorted(set(range(vocab_size)) - set(trained_tokens))

print(f"✓ Identified token usage")
print(f"  Trained: {len(trained_tokens)} tokens")
print(f"  Untrained (black hole): {len(untrained_tokens)} tokens")

untrained_indices = torch.tensor(untrained_tokens, dtype=torch.long)


Analyzing corpus: ../data/gatsby_clean.txt

✓ Identified token usage
  Trained: 79 tokens
  Untrained (black hole): 49 tokens


## Check Logit Equality for Untrained Tokens

In [5]:
print(f"\n{'='*80}")
print(f"LOGIT ANALYSIS")
print(f"{'='*80}\n")

for step in STEPS_TO_CHECK:
    if step >= n_steps:
        continue
    
    # Get logits for untrained tokens at this step
    untrained_logits = logits[step, untrained_indices]
    
    # Compute statistics
    logit_mean = untrained_logits.mean().item()
    logit_std = untrained_logits.std().item()
    logit_range = (untrained_logits.max() - untrained_logits.min()).item()
    
    print(f"Step {step:5d}:")
    print(f"  Untrained logits: mean={logit_mean:+.6f}, std={logit_std:.6e}, range={logit_range:.6e}")
    
    if logit_std < 1e-6:
        print(f"  → Logits are IDENTICAL (std < 1e-6)")
    elif logit_std < 1e-3:
        print(f"  → Logits are nearly identical (std < 1e-3)")
    else:
        print(f"  → Logits vary")
    print()

print(f"{'='*80}")


LOGIT ANALYSIS

Step     0:
  Untrained logits: mean=+0.000000, std=0.000000e+00, range=0.000000e+00
  → Logits are IDENTICAL (std < 1e-6)

Step     1:
  Untrained logits: mean=+0.632812, std=0.000000e+00, range=0.000000e+00
  → Logits are IDENTICAL (std < 1e-6)

Step     2:
  Untrained logits: mean=+1.039062, std=0.000000e+00, range=0.000000e+00
  → Logits are IDENTICAL (std < 1e-6)

Step    10:
  Untrained logits: mean=+0.457031, std=0.000000e+00, range=0.000000e+00
  → Logits are IDENTICAL (std < 1e-6)

Step   100:
  Untrained logits: mean=-2.609375, std=0.000000e+00, range=0.000000e+00
  → Logits are IDENTICAL (std < 1e-6)

Step  1000:
  Untrained logits: mean=-3.421875, std=0.000000e+00, range=0.000000e+00
  → Logits are IDENTICAL (std < 1e-6)

Step 10000:
  Untrained logits: mean=-2.921875, std=0.000000e+00, range=0.000000e+00
  → Logits are IDENTICAL (std < 1e-6)



## Check Gradient Equality for Untrained Tokens

In [6]:
print(f"\n{'='*80}")
print(f"GRADIENT ANALYSIS")
print(f"{'='*80}\n")

for step in STEPS_TO_CHECK:
    if step >= n_steps or step == 0:  # No gradients at step 0
        continue
    
    # Get gradients for untrained tokens at this step
    untrained_grads = gradients[step, untrained_indices, :]  # (n_untrained, hidden_dim)
    
    # Check if all gradients are identical by comparing pairwise distances
    # If identical, all pairwise distances should be 0
    pairwise_dists = torch.cdist(untrained_grads, untrained_grads, p=2)
    max_pairwise_dist = pairwise_dists.max().item()
    mean_pairwise_dist = pairwise_dists.mean().item()
    
    # Also check gradient magnitudes
    grad_norms = torch.norm(untrained_grads, p=2, dim=1)
    grad_norm_mean = grad_norms.mean().item()
    grad_norm_std = grad_norms.std().item()
    
    print(f"Step {step:5d}:")
    print(f"  Gradient norms: mean={grad_norm_mean:.6e}, std={grad_norm_std:.6e}")
    print(f"  Pairwise distances: max={max_pairwise_dist:.6e}, mean={mean_pairwise_dist:.6e}")
    
    if max_pairwise_dist < 1e-6:
        print(f"  → Gradients are IDENTICAL (max dist < 1e-6)")
    elif max_pairwise_dist < 1e-3:
        print(f"  → Gradients are nearly identical (max dist < 1e-3)")
    else:
        print(f"  → Gradients vary")
    print()

print(f"{'='*80}")


GRADIENT ANALYSIS

Step     1:
  Gradient norms: mean=4.576357e-02, std=0.000000e+00
  Pairwise distances: max=0.000000e+00, mean=0.000000e+00
  → Gradients are IDENTICAL (max dist < 1e-6)

Step     2:
  Gradient norms: mean=5.894304e-02, std=0.000000e+00
  Pairwise distances: max=0.000000e+00, mean=0.000000e+00
  → Gradients are IDENTICAL (max dist < 1e-6)

Step    10:
  Gradient norms: mean=5.045750e-02, std=0.000000e+00
  Pairwise distances: max=0.000000e+00, mean=0.000000e+00
  → Gradients are IDENTICAL (max dist < 1e-6)

Step   100:
  Gradient norms: mean=1.788655e-03, std=2.352435e-10
  Pairwise distances: max=0.000000e+00, mean=0.000000e+00
  → Gradients are IDENTICAL (max dist < 1e-6)

Step  1000:
  Gradient norms: mean=4.876784e-04, std=0.000000e+00
  Pairwise distances: max=0.000000e+00, mean=0.000000e+00
  → Gradients are IDENTICAL (max dist < 1e-6)

Step 10000:
  Gradient norms: mean=4.253804e-04, std=2.940543e-11
  Pairwise distances: max=0.000000e+00, mean=0.000000e+00
 

## Check Position Equality for Untrained Tokens

In [7]:
print(f"\n{'='*80}")
print(f"POSITION ANALYSIS")
print(f"{'='*80}\n")

for step in STEPS_TO_CHECK:
    if step >= n_steps:
        continue
    
    # Get positions for untrained tokens at this step
    untrained_positions = embeddings[step, untrained_indices, :]  # (n_untrained, hidden_dim)
    
    # Check if all positions are identical
    pairwise_dists = torch.cdist(untrained_positions, untrained_positions, p=2)
    max_pairwise_dist = pairwise_dists.max().item()
    mean_pairwise_dist = pairwise_dists.mean().item()
    
    print(f"Step {step:5d}:")
    print(f"  Position pairwise distances: max={max_pairwise_dist:.6e}, mean={mean_pairwise_dist:.6e}")
    
    if max_pairwise_dist < 1e-6:
        print(f"  → Positions are IDENTICAL (all tokens at same point)")
    elif max_pairwise_dist < 1e-3:
        print(f"  → Positions are nearly identical (tight cluster)")
    else:
        print(f"  → Positions vary (tokens spread out)")
    print()

print(f"{'='*80}")


POSITION ANALYSIS

Step     0:
  Position pairwise distances: max=0.000000e+00, mean=0.000000e+00
  → Positions are IDENTICAL (all tokens at same point)

Step     1:
  Position pairwise distances: max=6.103516e-05, mean=6.103516e-05
  → Positions are nearly identical (tight cluster)

Step     2:
  Position pairwise distances: max=0.000000e+00, mean=0.000000e+00
  → Positions are IDENTICAL (all tokens at same point)

Step    10:
  Position pairwise distances: max=4.315837e-05, mean=4.315836e-05
  → Positions are nearly identical (tight cluster)

Step   100:
  Position pairwise distances: max=0.000000e+00, mean=0.000000e+00
  → Positions are IDENTICAL (all tokens at same point)

Step  1000:
  Position pairwise distances: max=0.000000e+00, mean=0.000000e+00
  → Positions are IDENTICAL (all tokens at same point)

Step 10000:
  Position pairwise distances: max=0.000000e+00, mean=0.000000e+00
  → Positions are IDENTICAL (all tokens at same point)



## Summary

In [8]:
print(f"\n{'='*80}")
print(f"SUMMARY")
print(f"{'='*80}\n")
print(f"Hypothesis: Black hole moves because untrained tokens receive identical gradients.")
print()
print(f"Causal chain:")
print(f"  1. Tokens at same position → identical when dotted with hidden state")
print(f"  2. Identical dot products → identical logits")
print(f"  3. Identical logits → identical gradients (after loss backward)")
print(f"  4. Identical gradients → tokens move together (stay coincident)")
print(f"  5. Still coincident → cycle repeats")
print()
print(f"Result: Black hole moves as a RIGID BODY through embedding space.")
print()
print(f"Check the analysis above to verify:")
print(f"  ✓ Positions identical? (supermassive BH = all at one point)")
print(f"  ✓ Logits identical? (same position → same score)")
print(f"  ✓ Gradients identical? (same score → same update)")
print()
print(f"Implication for thermodynamics:")
print(f"  The black hole HAS NO INTERNAL MOTION.")
print(f"  It moves as a single object, not a gas of particles.")
print(f"  Temperature of black hole = 0 (no relative motion).")
print()
print(f"{'='*80}")


SUMMARY

Hypothesis: Black hole moves because untrained tokens receive identical gradients.

Causal chain:
  1. Tokens at same position → identical when dotted with hidden state
  2. Identical dot products → identical logits
  3. Identical logits → identical gradients (after loss backward)
  4. Identical gradients → tokens move together (stay coincident)
  5. Still coincident → cycle repeats

Result: Black hole moves as a RIGID BODY through embedding space.

Check the analysis above to verify:
  ✓ Positions identical? (supermassive BH = all at one point)
  ✓ Logits identical? (same position → same score)
  ✓ Gradients identical? (same score → same update)

Implication for thermodynamics:
  The black hole HAS NO INTERNAL MOTION.
  It moves as a single object, not a gas of particles.
  Temperature of black hole = 0 (no relative motion).

